In [13]:
from libs.alibaba.alibaba import Alibaba

from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import html
import time
import re
import traceback
import types

In [4]:
lid = 'eyelash@glittergroupcn.com'
lpwd = 'Wangzhen68970747'
alibaba = Alibaba(lid, lpwd)

In [5]:
alibaba.chrome_options.add_argument('--headless')

In [6]:
alibaba.login()
browser = alibaba.browser

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [21]:
alibaba.crawl_product_data('result_message_60771119046', str(60771026019))

primary 打开产品 [60771026019] 编辑页面 ... ...
primary 开始爬取产品 [60771026019] 数据 ... ...
success 完成 对产品 [60771026019] 编辑页面 的数据爬取！
result_message_60771119046 {'attributes': {'alibaba_id': '60771026019', 'alibaba_category': 'Beauty & Personal Care>>Makeup Tools>>False Eyelashes', 'name': 'Private Label 3D Lace Paper and silk lashes curling False Eyelashes for party', 'keywords': ['wholesale eyelashes', 'free eyelashes samples', 'custom eyelashes'], 'type': '21B05', 'brand': 'GLITTER', 'origin': ['China (Mainland)', 'Shandong'], 'material': 'Synthetic Hair', 'production_method': 'Hand Made', 'custom_attributes': [['Product Name', 'Top Quality Grade 100% Handmade Cotton Band 3D Silk False Eyelashes'], ['Color', 'Black'], ['Price', 'Factory Direct'], ['Sizes', 'All sizes are in stock'], ['Grade', 'Top grade, soft and natural looking'], ['Sample', 'Free Sample'], ['OEM/ODM', 'Accepted'], ['Assurance', 'Alibaba Trade Assurance'], ['Inspection', '100% Before Shipment'], ['Custom Packaging', 'Accepted']

True

In [19]:
def crawl_product_data(self, result_message, ali_id):
    api = 'https://hz-productposting.alibaba.com/product/editing.htm?id='
    browser = self.browser
    if not browser:
        self.notify("danger", "没有登录，请先登录")
        return
    attrs = {'alibaba_id': ali_id}
    template = {}
    try:
        self.notify("primary", "打开产品 [" + ali_id + "] 编辑页面 ... ...")
        browser.get(api + ali_id)
        WebDriverWait(browser, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#editor-container')))

        self.notify("primary", "开始爬取产品 [" + ali_id + "] 数据 ... ...")
        attrs['alibaba_category'] = browser.find_element_by_css_selector('span#cate-path-text-copy').text.strip()
        attrs['name'] = browser.find_element_by_css_selector('input#productName').get_attribute('value').strip()
        keywords = []
        keyword = browser.find_element_by_css_selector('input#productKeyword').get_attribute('value').strip()
        keywords.append(keyword)
        keyword = browser.find_element_by_css_selector('input#keywords2').get_attribute('value').strip()
        keywords.append(keyword)
        keyword = browser.find_element_by_css_selector('input#keywords3').get_attribute('value').strip()
        keywords.append(keyword)
        attrs['keywords'] = keywords
        attrs['type'] = browser.find_element_by_css_selector(
            '#productAttribute>div:first-child>div>input').get_attribute('value').strip()
        attrs['brand'] = browser.find_element_by_css_selector(
            '#productAttribute>div:nth-child(2)>div>input').get_attribute('value').strip()

        origin = []
        select = Select(browser.find_elements_by_css_selector('#productAttribute>div:nth-child(3) select')[0])
        origin.append(select.first_selected_option.text)

        select = Select(browser.find_elements_by_css_selector('#productAttribute>div:nth-child(3) select')[1])
        origin.append(select.first_selected_option.text)
        attrs['origin'] = origin

        select = Select(browser.find_elements_by_css_selector('#productAttribute>div:nth-child(4) select')[0])
        attrs['material'] = select.first_selected_option.text

        select = Select(browser.find_elements_by_css_selector('#productAttribute>div:nth-child(5) select')[0])
        attrs['production_method'] = select.first_selected_option.text

        custom_attributes = []
        trs = browser.find_elements_by_css_selector('#attr table tr')
        trs.pop(0)
        for tr in trs:
            key = tr.find_elements_by_css_selector('input')[0].get_attribute('value').strip()
            value = tr.find_elements_by_css_selector('input')[1].get_attribute('value').strip()
            custom_attributes.append([key, value])
        attrs['custom_attributes'] = custom_attributes

        browser.find_element_by_css_selector('#mceu_0 button').click()
        WebDriverWait(browser, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mceu_46 textarea')))
        time.sleep(1)
        template['html'] = browser.find_element_by_css_selector('#mceu_46 textarea').get_attribute('value').strip()
        browser.find_element_by_css_selector('#mceu_0 button').click()
        WebDriverWait(browser, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mceu_46 iframe')))
        time.sleep(1)

        css_selector = '#tradInfoblock>div:nth-child(3) .price-scroll-selector input[value="uniform"]'
        element = browser.find_element_by_css_selector(css_selector)
        attrs['isTieredPricing'] = element.is_selected()
        if attrs['isTieredPricing']:
            attrs['sales_unit'] = browser.find_element_by_css_selector('#tradeInfoQuantityUnit').text.strip()
            attrs['currency'] = 'USD'
            tieredPricing = []
            trs = browser.find_elements_by_css_selector('#priceSetter .next-table-body tbody tr')
            for tr in trs:
                volume = tr.find_elements_by_css_selector('input')[0].get_attribute('value').strip()
                price = tr.find_elements_by_css_selector('input')[1].get_attribute('value').strip()
                tieredPricing.append([volume, price])
            attrs['tieredPricing'] = tieredPricing
        else:
            attrs['currency'] = browser.find_element_by_css_selector('#moneyTypeSelect span').text.strip()
            price_range = []
            price_range.append(
                browser.find_element_by_css_selector('#priceRangeMin').get_attribute('value').strip())
            price_range.append(
                browser.find_element_by_css_selector('#priceRangeMax').get_attribute('value').strip())
            attrs['price_range'] = price_range
            attrs['sales_unit'] = browser.find_element_by_css_selector('#priceUnitSelect span').text.strip()
            attrs['MOQ'] = browser.find_element_by_css_selector('#minOrderQuantity').get_attribute('value').strip()
            attrs['additional_trading_infomation'] = browser.find_element_by_css_selector(
                '#minOrderOther').get_attribute('value').strip()
        payment_methods = {}
        checkboxes = browser.find_elements_by_css_selector('.trade-info-payment-group input[type="checkbox"]')
        for cb in checkboxes:
            value = cb.get_attribute('value')
            checked = cb.is_selected()
            if value == "Others":
                continue
            payment_methods[value] = checked
        attrs['payment_methods'] = payment_methods
        if browser.find_element_by_css_selector('#paymentMethodOther').is_selected():
            attrs['other_payment_method'] = browser.find_element_by_css_selector(
                '#paymentMethodOtherDesc').get_attribute('value')

        attrs['delivery_time'] = browser.find_element_by_css_selector('#consignmentTerm').get_attribute('value')
        attrs['port'] = browser.find_element_by_css_selector('#port').get_attribute('value')
        supply_ability = []
        supply_ability.append(browser.find_element_by_css_selector('#supplyQuantity').get_attribute('value'))
        select = Select(browser.find_element_by_css_selector('#supplyUnit'))
        supply_ability.append(select.first_selected_option.text)
        select = Select(browser.find_element_by_css_selector('#supplyPeriod'))
        supply_ability.append(select.first_selected_option.text)
        attrs['supply_ability'] = supply_ability
        attrs['additional_delivery_information'] = browser.find_element_by_css_selector(
            '#supplyOther').get_attribute('value')
        attrs['packaging'] = browser.find_element_by_css_selector('#packagingDesc').get_attribute('value')

        self.notify('success', "完成 对产品 [" + ali_id + "] 编辑页面 的数据爬取！")
    except TimeoutException as e:
        attrs = None
        template = None
        msg = "爬取产品 [" + ali_id + '] 数据 出错，超时，' + str(e)
    except NoSuchElementException as e:
        attrs = None
        template = None
        msg = "爬取产品 [" + ali_id + "] 数据 出错，没找到目标元素, " + str(e)
    except Exception as e:
        attrs = None
        template = None
        msg = "爬取产品 [" + ali_id + "] 数据 出错, " + str(e)
        browser.save_screenshot('screenie.png')
        traceback.print_exc()
    finally:
        browser.get('https://i.alibaba.com')

    if not attrs or not template:
        self.notify("danger", msg)
        self.notify(result_message, None)
    else:
        result = {'attributes': attrs, 'template': template}
        self.notify(result_message, result)
alibaba.crawl_product_data = types.MethodType(crawl_product_data, alibaba)

primary 打开产品 [60763545453] 编辑页面 ... ...
{'ali_id': 60763545453, 'price': {'isTieredPricing': False, 'price_range': ['3.3', '4.6'], 'tieredPricing': []}, 'detail_pictures': [{'old': '//sc02.alicdn.com/kf/HTB156nci3KTBuNkSne1q6yJoXXaO/231186930/HTB156nci3KTBuNkSne1q6yJoXXaO.jpg', 'new': '//sc02.alicdn.com/kf/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT/231186930/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT.jpg'}, {'old': '//sc02.alicdn.com/kf/HTB1KAzci3KTBuNkSne1q6yJoXXas/231186930/HTB1KAzci3KTBuNkSne1q6yJoXXas.jpg', 'new': '//sc01.alicdn.com/kf/HTB1sJ4Ox25TBuNjSspmq6yDRVXax/231186930/HTB1sJ4Ox25TBuNjSspmq6yDRVXax.jpg'}, {'old': '//sc01.alicdn.com/kf/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY/231186930/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY.jpg', 'new': '//sc02.alicdn.com/kf/HTB176d8x49YBuNjy0Ffq6xIsVXaE/231186930/HTB176d8x49YBuNjy0Ffq6xIsVXaE.jpg'}]}
primary 设置产品 [60763545453] 的价格区间 ... ...


primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [1]:
pictures = []
picture = {'old': "//sc02.alicdn.com/kf/HTB156nci3KTBuNkSne1q6yJoXXaO/231186930/HTB156nci3KTBuNkSne1q6yJoXXaO.jpg"}
picture['new'] = "//sc02.alicdn.com/kf/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT/231186930/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT.jpg"
pictures.append(picture)
picture = {'old': "//sc02.alicdn.com/kf/HTB1KAzci3KTBuNkSne1q6yJoXXas/231186930/HTB1KAzci3KTBuNkSne1q6yJoXXas.jpg"}
picture['new'] = "//sc01.alicdn.com/kf/HTB1sJ4Ox25TBuNjSspmq6yDRVXax/231186930/HTB1sJ4Ox25TBuNjSspmq6yDRVXax.jpg"
pictures.append(picture)
picture = {'old': "//sc01.alicdn.com/kf/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY/231186930/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY.jpg"}
picture['new'] = "//sc02.alicdn.com/kf/HTB176d8x49YBuNjy0Ffq6xIsVXaE/231186930/HTB176d8x49YBuNjy0Ffq6xIsVXaE.jpg"
pictures.append(picture)
price = {'isTieredPricing': False, 'price_range': [3.3, 4.8], 'tieredPricing': []}
data = dict(ali_id="60764140968", price=price, detail_pictures=pictures)

In [94]:
alibaba.update_product(data)

primary 打开产品 [60764140968] 编辑页面 ... ...
primary 设置产品 [60764140968] 的价格区间 ... ...


In [2]:
data

{'ali_id': '60764140968',
 'price': {'isTieredPricing': False,
  'price_range': [3.3, 4.8],
  'tieredPricing': []},
 'detail_pictures': [{'old': '//sc02.alicdn.com/kf/HTB156nci3KTBuNkSne1q6yJoXXaO/231186930/HTB156nci3KTBuNkSne1q6yJoXXaO.jpg',
   'new': '//sc02.alicdn.com/kf/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT/231186930/HTB1cn7DpvuSBuNkHFqDq6xfhVXaT.jpg'},
  {'old': '//sc02.alicdn.com/kf/HTB1KAzci3KTBuNkSne1q6yJoXXas/231186930/HTB1KAzci3KTBuNkSne1q6yJoXXas.jpg',
   'new': '//sc01.alicdn.com/kf/HTB1sJ4Ox25TBuNjSspmq6yDRVXax/231186930/HTB1sJ4Ox25TBuNjSspmq6yDRVXax.jpg'},
  {'old': '//sc01.alicdn.com/kf/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY/231186930/HTB1xBa5iZuYBuNkSmRyq6AA3pXaY.jpg',
   'new': '//sc02.alicdn.com/kf/HTB176d8x49YBuNjy0Ffq6xIsVXaE/231186930/HTB176d8x49YBuNjy0Ffq6xIsVXaE.jpg'}]}

In [89]:
element = browser.find_element_by_css_selector("#editor-container")
ActionChains(browser).move_to_element(element).perform()
# self.notify("primary", "开始 设置 产品详情页 ... ...")
# 点击 详情编辑器 ‘源代码’ 按钮， 清空内容，再切换回 富文本 编辑状态
browser.find_element_by_css_selector('#mceu_0 button').click()
textarea = WebDriverWait(browser, 5).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, '#mceu_46 textarea')))

In [90]:
text = textarea.get_attribute('value')
soup = BeautifulSoup(text, 'html.parser')
for img in soup.find_all('img'):
    src = img['src']
    key = re.sub('//[^\.]*\.', '', src)
    
    for idx, pic in enumerate(pictures):
        if key not in pic['old']:
            continue
            
        img['src'] = pic['new']
        if img.has_attr('ori-width'):
            del img['ori-width']
        if img.has_attr('ori-height'):
            del img['ori-height']
        break


In [91]:

js_templ = "document.querySelector('{selector}').value = '{value}';"
js = js_templ.format(selector='#mceu_46 textarea', value=re.sub('\n', '\\\\n', str(soup)))
browser.find_element_by_css_selector('#mceu_46 textarea').clear()
browser.execute_script(js)
browser.find_element_by_css_selector('#mceu_0 button').click()
WebDriverWait(browser, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mceu_46 iframe')))